# 1. Installation and imports

In [1]:
#Install the required librairies

#Pip upgrade
!pip install --upgrade pip 

#Model fine-tuning and model saving
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!python3 -m pip install transformers
!python3 -m pip install sacremoses
!python3 -m pip install fastbpe
!python3 -m pip install pyyaml
!python3 -m pip install folium==0.2.1
!python3 -m pip install datasets
!python3 -m pip install SentencePiece

#Lime 
!python3 -m pip install lime

#Shap
!python3 -m pip install shap

#LRP 
!python3 -m pip install -r Transformer-Explainability/requirements.txt
!python3 -m pip install captum

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'Transformer-Explainability/requirements.txt'


In [2]:
#Basic imports
import os
import random
import numpy as np
import copy as cp
import sys
from tqdm import tqdm

#Dataset imports
from transformers import DataCollatorWithPadding
import pandas as pd 
import datasets
import re

#Model fine-tuning imports
import torch
from torch.utils.data import DataLoader 
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup
import time
import datetime

#Explainer imports
from lime.lime_text import LimeTextExplainer
import shap

from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import CamembertForSequenceClassification
from captum.attr import visualization


print(torch.__version__)
print(torch.cuda.get_arch_list())

1.12.1+cu116
['sm_37', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'sm_80', 'sm_86']


In [3]:
#Set up device
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

## 1.1 Execution flags

In [4]:
TRAIN_MODEL = True# Whether the model should be train or not
LOAD_MODEL = not TRAIN_MODEL # Whether the model should be load or not  (define as the opposite of TRAIN_MODEL)
EVALUATE = False # Whether we should evalute the performances of the model on the 3 sets or not
VISUALIZE = False # Whether we should visualize the explanations or not
DO_LIME = False # Whether we should compute the lime explanations or not
DO_SHAP = False # Whether we should compute the lime explanations or not
DO_LRP = False # Whether we should compute the lime explanations or not
DO_LING = False # Whether we should compute the lime explanations or not

# 2. Dataset preprocessing

In [5]:
def preprocess(text):
    
    #1 Replace the apostrophes
    apo_regex = "(\W[cçdjlmnst]|\Wqu|[CÇDJLMNST]|Qu)[’'ʼ]"
    characters = re.findall(apo_regex, text)
    new_text = re.sub(apo_regex, " <APO0> ", text)

    #2 Replace the inword apostrophes (ex: aujourd'hui)
    word_apo_regex = "(?<=[a-z])[’'ʼ](?=[a-z])"
    new_text = re.sub(word_apo_regex, " <APO1> ", new_text)
    
    #3 Replace all the other ones by quotes
    quote_regex = "['\"’“”«»‘’ʼ]"
    new_text = re.sub(quote_regex, " <QUOTE> ", new_text)
    
    #4 Replace the numbers
    nbr_regex = "(?<=[^\w])[0-9]+(?=[^\w])|(?<=[hH])[0-9]+|[0-9]+(?=[hH])"
    new_text = re.sub(nbr_regex, "<NBR>", new_text)
    
    flag = True
    comp_nbr_regex = "<NBR>[., ]?<NBR>"
    while flag:
        flag = False
        old_text = new_text
        new_text = re.sub(comp_nbr_regex, "<NBR>", new_text)
        flag = (old_text != new_text)
    
    new_text = re.sub("<NBR>", " NBR ", new_text)
    
    new_text = re.sub('(?<=\w| )[.](?=[^\.]|$)', ' . ', new_text)
    new_text = re.sub('\]', ' ] ', new_text)
    new_text = re.sub('\[', ' [ ', new_text)
    
    new_text = re.sub('\?', ' ? ', new_text)
    new_text = re.sub('\!', ' ! ', new_text)
    new_text = re.sub('\,', ' , ', new_text)
    new_text = re.sub('\;', ' ; ', new_text)
    new_text = re.sub('\(', ' ( ', new_text)
    new_text = re.sub('\)', ' ) ', new_text)
    new_text = re.sub('\.\.\.|…', ' ... ', new_text)
    new_text = re.sub(':', ' : ', new_text)
    
    character_pos = 0
    clean_text = ""
    space_next=True
    for elem in new_text.split():
        if elem.strip() == "<APO0>":
            clean_text += " " + characters[character_pos].strip() + "'"
            character_pos += 1
        elif elem.strip() == "<APO1>":
            clean_text += "'"
            space_next = False
        elif elem.strip() == "<QUOTE>":
            clean_text += ' "'
        else:
            if space_next:
                clean_text += " "
            space_next = True
            clean_text += elem.strip()

    assert character_pos == len(characters)
    return clean_text.strip()

In [6]:
dataDfInit = pd.read_csv("../NeededFiles/dataset/rtbf_dataset_14000.csv")
dataDfInit = dataDfInit.rename(columns={"article": "text"})
dataDfInit["text"] = dataDfInit["text"].apply(preprocess)

In [7]:
print(dataDfInit["text"][3])

The Mirror , qui n' a jamais été fan de Johnson , l' appelle le plus mauvais Premier ministre de Grande-Bretagne ( depuis le dernier ) " - en précisant clairement à qui ils font référence avec une photo incrustée de Theresa May . Selon le journal , Johnson est devenu " le seul Premier ministre à avoir perdu ses trois premiers votes à la Chambre des communes , les députés ayant opposé son veto à sa stratégie risquée de non-accord " . "


In [8]:
df_actu = dataDfInit[dataDfInit.category == 'actu']
df_chron = dataDfInit[dataDfInit.category == 'chron']

df_chron = df_chron.reset_index()
df_chron = df_chron.loc[df_chron.index <= 5999]
df_chron['label'] = 1
df_chron = df_chron[['text', 'label']]

df_actu = df_actu.reset_index()
df_actu = df_actu.loc[df_actu.index <= 5999]
df_actu['label'] = 0
df_actu = df_actu[['text', 'label']] 

In [9]:
train_chron = df_chron[df_chron.index < 4800]
val_chron = df_chron[(df_chron.index >= 4800) & (df_chron.index < 5400)]
test_chron = df_chron[(df_chron.index >= 5400)]

train_actu = df_actu[df_actu.index < 4800]
val_actu = df_actu[(df_actu.index >= 4800) & (df_actu.index < 5400)]
test_actu = df_actu[(df_actu.index >= 5400)]

train_df = pd.concat([train_chron, train_actu], axis=0)
val_df = pd.concat([val_chron, val_actu], axis=0)
test_df = pd.concat([test_chron, test_actu], axis=0)

seed = 42
FRAC = 1

train_ds = datasets.Dataset.from_pandas(train_df.sample(frac=FRAC, random_state = seed))
val_ds = datasets.Dataset.from_pandas(val_df.sample(frac=FRAC, random_state = seed))
test_ds = datasets.Dataset.from_pandas(test_df.sample(frac=FRAC, random_state = seed))

rtbf_dataset = datasets.DatasetDict({"train":train_ds,"val":val_ds, "test": test_ds})

In [10]:
print(len(train_df), len(val_df), len(test_df))

9600 1200 1200


# 2. Model Loading

In [11]:
modelname = 'camembert-base' 
do_lowercase = ("cased" not in modelname)

modelType = "Camembert_punct" if LOAD_MODEL else "Camembert_base"

if modelType == "Camembert_base":
    # Load pretrained model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained(modelname)
    tokenizer = AutoTokenizer.from_pretrained(modelname, do_lowercase=do_lowercase)#, additional_special_tokens = ["<NBR>"])

elif modelType == "Camembert_trained":
    # Load trained model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained("../NeededFiles/camembert_model")
    tokenizer = AutoTokenizer.from_pretrained("../NeededFiles/camembert_model", do_lowercase=do_lowercase)
    
elif modelType == "Camembert_punct":
    # Load trained model and tokenizer trained on preprocessed input
    model = AutoModelForSequenceClassification.from_pretrained("../NeededFiles/camembert_model_punct")
    tokenizer = AutoTokenizer.from_pretrained("../NeededFiles/camembert_model_punct", do_lowercase=do_lowercase)

else:
    print("Unknown modeltype, choose between Camembert_base, Camembert_trained and Camembert_punct")
    print("Exiting")
    sys.exit(0)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 

In [12]:
model.to(device)

CamembertForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Laye

## 2.1 Tokenization and dataloader creation

In [13]:
max_length = 512
batch_size = 4

def preprocess_function(examples):
    return tokenizer(examples["text"], max_length = max_length, padding = 'max_length', truncation=True, return_tensors='pt')

tokenized_rtbf_full = rtbf_dataset.map(preprocess_function, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [14]:
train_text = tokenized_rtbf_full["train"]["text"]
val_text = tokenized_rtbf_full["val"]["text"]
test_text = tokenized_rtbf_full["test"]["text"]

In [15]:
print(test_text[3])

LA CITOYENNETE ET LES TIC Fête de la Communauté Française oblige , vous vous intéressez ce matin à lapport des nouvelles technos à la citoyenneté . Oui . Les discours de circonstance vont sans doute aborder le destin des francophones de ce pays . Cest le moment de rappeler que la citoyenneté nest pas quune question didentité . Elle sentretient dans la participation à la vie collective . Pour ça , il faut que lon soit informé et que lon puisse communiquer . Et aujourdhui , les TIC , les technologies de linformation et la communication peuvent vraiment y contribuer . Bien sûr , cest vaste . Le-government , la gouvernance électronique , couvre des domaines extrêmement variés . Dabord administratifs . La plus connue cest Tax-On-Web . Mais la carte didentité électronique permettrait daller beaucoup plus loin . Jusquà organiser des consultations via le web par exemple ! A condition davoir un vrai projet densemble axé sur le citoyen . Et pas un éparpillement dans un maquis de niveaux de pouvo

In [16]:
tokenized_rtbf = tokenized_rtbf_full.remove_columns(['text', '__index_level_0__'])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, max_length = max_length, padding = 'max_length')

train = DataLoader(tokenized_rtbf["train"], 
                   shuffle=False,
                   batch_size=batch_size,
                   collate_fn=data_collator)

val = DataLoader(tokenized_rtbf["val"], 
                   shuffle=False,
                   batch_size=batch_size,
                   collate_fn=data_collator)

test = DataLoader(tokenized_rtbf["test"], 
                   shuffle=False,
                   batch_size=batch_size,
                   collate_fn=data_collator)

## 2.1 Model training (Optional)

In [17]:
# Optimizer creation 

optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data a bit.
epochs = 2
batches_per_epoch = len(train) // batch_size
# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = int(batches_per_epoch * epochs)

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [18]:
# Utils functions used during the training

def flat_accuracy(preds, labels):
        pred_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [19]:
if TRAIN_MODEL:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # We'll store a number of quantities such as training and validation loss, 
    # validation accuracy, and timings.
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()

    # For each epoch...
    for epoch_i in range(0, epochs):

        # ========================================
        #               Training
        # ========================================

        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0

        # Put the model into training mode. Don't be mislead--the call to 
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(tqdm(train, total=len(train))):

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the 
            # `to` method.
            #
            cp_batch = cp.deepcopy(batch)
            b_input_ids = cp_batch['input_ids'].to(device)
            b_input_mask = cp_batch['attention_mask'].to(device)
            b_labels = cp_batch['labels'].to(device)

            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because 
            # accumulating the gradients is "convenient while training RNNs". 
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()        

            # Perform a forward pass (evaluate the model on this training batch).
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # It returns different numbers of parameters depending on what arguments
            # arge given and what flags are set. For our useage here, it returns
            # the loss (because we provided labels) and the "logits"--the model
            # outputs prior to activation.

            loss, logits = model(b_input_ids, 
                                 token_type_ids=None, 
                                 attention_mask=b_input_mask, 
                                 labels=b_labels)[:2]




            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_train_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train)            

        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables 
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        # Evaluate data for one epoch
        for step, batch in enumerate(tqdm(val, total=len(val))):

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the 
            # `to` method.
            #
            cp_batch = cp.deepcopy(batch)
            b_input_ids = cp_batch['input_ids'].to(device)
            b_input_mask = cp_batch['attention_mask'].to(device)
            b_labels = cp_batch['labels'].to(device)

            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():        

                # Forward pass, calculate logit predictions.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # Get the "logits" output by the model. The "logits" are the output
                # values prior to applying an activation function like the softmax.
                loss, logits = model(b_input_ids, 
                                        token_type_ids=None, 
                                        attention_mask=b_input_mask,
                                        labels=b_labels)[:2]

            # Accumulate the validation loss.
            total_eval_loss += loss.item()

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_eval_accuracy += flat_accuracy(logits, label_ids)


        # Report the final accuracy for this validation run.
        avg_val_accuracy = total_eval_accuracy / len(val)
        print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(val)

        # Measure how long the validation run took.
        validation_time = format_time(time.time() - t0)

        print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...


100%|███████████████████████████████████████| 2400/2400 [04:54<00:00,  8.14it/s]



  Average training loss: 0.14
  Training epcoh took: 0:04:55

Running Validation...


100%|█████████████████████████████████████████| 300/300 [00:10<00:00, 27.40it/s]


  Accuracy: 0.98
  Validation Loss: 0.09
  Validation took: 0:00:11

======== Epoch 2 / 2 ========
Training...


100%|███████████████████████████████████████| 2400/2400 [04:57<00:00,  8.07it/s]



  Average training loss: 0.10
  Training epcoh took: 0:04:57

Running Validation...


100%|█████████████████████████████████████████| 300/300 [00:11<00:00, 26.76it/s]

  Accuracy: 0.98
  Validation Loss: 0.09
  Validation took: 0:00:11

Training complete!
Total training took 0:10:15 (h:mm:ss)


In [20]:
if TRAIN_MODEL:
    # ========================================
    #               Test
    # ========================================
    # After the whola training measure our performance on our test set.

    print("")
    print("Running Test...")
    import time 
    import copy as cp
    t0 = time.time()
    total_eval_accuracy = 0
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    predictions , true_labels = [], []

    # Evaluate data for one epoch
    for step, batch in enumerate(tqdm(test, total = len(test))):

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        cp_batch = cp.deepcopy(batch)
        b_input_ids = cp_batch['input_ids'].to(device)
        b_input_mask = cp_batch['attention_mask'].to(device)
        b_labels = cp_batch['labels'].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            outputs = model(b_input_ids, token_type_ids=None, 
                          attention_mask=b_input_mask)

        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)

        # Store predictions and true labels
        predictions.append(logits)
        true_labels.append(label_ids)


    avg_test_accuracy = total_eval_accuracy / len(test)
    print("  Accuracy: {0:.4f}".format(avg_test_accuracy))


Running Test...


100%|█████████████████████████████████████████| 300/300 [00:11<00:00, 26.75it/s]

  Accuracy: 0.9783


In [21]:
if TRAIN_MODEL:
    # Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

    #output_dir = '../NeededFiles/camembert_model_punct/'
    output_dir = '../NeededFiles/camembert_model_punct/'
    # Create output directory if needed
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print("Saving model to %s" % output_dir)

    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    model = AutoModelForSequenceClassification.from_pretrained("../NeededFiles/camembert_model_punct")
    tokenizer = AutoTokenizer.from_pretrained("../NeededFiles/camembert_model_punct", do_lowercase=do_lowercase)
    model.to(device)

Saving model to ../NeededFiles/camembert_model_punct/


## 2.2 Model evaluation

In [22]:
#Function to compute the accuracy after the training

def accuracy_pt(model, dataset, withGrad=False): 

    total_accuracy = 0
    total_loss = 0
    model.eval()

    for step, batch in enumerate(tqdm(dataset, total=len(dataset))):
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['labels'].to(device)

        if withGrad:
          loss, logits = model(b_input_ids, 
                               token_type_ids=None, 
                               attention_mask=b_input_mask,
                               labels=b_labels)[:2]
        
        else:
          with torch.no_grad():        

            loss, logits = model(b_input_ids, 
                                 token_type_ids=None, 
                                 attention_mask=b_input_mask,
                                 labels=b_labels)[:2]
        
        total_loss += loss.item()
        loss.detach().cpu()
        
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.detach().cpu().numpy()
        
        b_input_ids.detach().cpu()
        b_input_mask.detach().cpu()
        b_labels.detach().cpu()
        
        total_accuracy += flat_accuracy(logits, label_ids)

    avg_accuracy = total_accuracy / len(dataset)
    avg_loss = total_loss / len(dataset)
    return avg_accuracy, avg_loss

In [23]:
EVALUATE = False
if EVALUATE: 
   print("Computing accuracies and loss for train, val and test sets")
   train_acc = accuracy_pt(model, train)
   print(f"Training accuracy: {train_acc[0]}, Training loss : {train_acc[1]}")
   validation_acc = accuracy_pt(model, val)
   print(f"Validation accuracy: {validation_acc[0]}, Validation loss : {validation_acc[1]}")
   test_acc = accuracy_pt(model, test)
   print(f"Test accuracy: {test_acc[0]}, Test loss : {test_acc[1]}")

# 3. Explainer part

In [51]:
text_samples = ["Premier chiffre : 242 milliards d’euros. Telle est la somme invraisemblable qui repose sur les comptes d’épargne en Belgique. Si on a la curiosité de diviser ce chiffre par dix millions de Belges, ça nous fait tout de même une moyenne de 24.000 € par personne. Si, comme moi, vous avez l’impression de connaître davantage d’individus en-dessous des 24.000 qu’au-dessus, sachez que vous faites partie de la mauvaise moyenne. De toute façon, bonne chance pour aborder le sujet : l’argent qu’on gagne, l’argent qu’on thésaurise est et restera le dernier tabou des Belges. Mais lorsque sortent ces chiffres, comme la semaine dernière, se pose logiquement la question de la manière dont cet argent pourrait être mobilisé et utilisé. C’est vrai que de l’argent fixe, ça n’arrange personne : ni l’Etat, qui se ponctionne lorsque l’argent circule par la TVA, l’IPP, l’ISOC, etc., ni les entreprises, ni même les particuliers, à qui il ne rapporte rien. Belga",
                "Ah… la Saint-Valentin. Journée annuelle appartenant aux amoureux. Temps merveilleux de ceux et celles qui ne conjuguent plus que par nous, qui s’aiment un peu, beaucoup, passionnément, à la folie. Douces heures que celles du 14 février, moment tendresse où l’on célèbre l’autre, son autre, sa moitié, son presque tout. Si les enthousiastes se presseront au-devant d’étalages tout de roses garnis, à l’affut d’une attention délicate ou d’un présent extravagant chargé d’en mettre plein la vue de l’être aimé, d’autres à l’inverse verront leurs poils se hérisser et s’offusqueront contre ce qu’ils considèreront être une manœuvre capitaliste à peine dissimulée.",
                "Les marques de solidarité, nombreuses et émouvantes, empruntent une voie abominablement familière ces derniers temps ; quant à la réaction des résidents belges, quant à leur manière d’exorciser l’horreur, elle induit également un effet de déjà-vu – les leitmotivs de Paris ne sont pas loin : ici aussi, on se replie sur des particularismes somme toute dérisoires, néanmoins exaltés, ici aussi, on se réfère à des bribes vaguement identitaires, à des motifs de fierté un peu insignifiants, mais auxquels on est soulagé de pouvoir se cramponner dans l’adversité : la bande dessinée, la bière, les frites, un goût pour l’humour et le surréalisme, un côté débonnaire et gai luron, une faculté innée, paraît-il, à goûter les plaisirs simples qu’offre la vie – ces mêmes plaisirs que les terroristes voudraient frapper d’anathème.",
                "Jan Jambon précise même qu’il n’a pas à prouver ce qu’il dit; il le sait et ça suffit.L’homme est un habitué. A peine en fonction, il déclarait: 'Les gens qui ont collaboré avec les Allemands avaient leurs raisons.' Déjà à l’époque (octobre 2014), il regrette l’exploitation de ses propos. Il aurait pu presque dire: 'Une part significative des nationalistes flamands justifie la collaboration…' Il en sera de même lorsqu’il affirmera vouloir 'nettoyer Molenbeek'. A chaque fois, le chef de file nationaliste au fédéral entend exprimer ce que son électorat pense tout bas… Et de jeter un œil du côté du Vlaams Belang. Philippe Walkowiak",
                "Le groupe État islamique accueille de nombreux combattants occidentaux. L’organisation joue sur ces jeunes radicalisés pour mener des attaques. Les attentats de Bruxelles ont montré une nouvelle fois l’étendue de la menace djihadiste. La réaction face au groupuscule djihadiste doit être globale. Sur le terrain, les opérations militaires doivent aller de pair avec des gestes politiques et humanitaires. Chez nous, les attaques de Bruxelles ont montré la nécessité d'une plus grande collaboration entre les forces de police et les services de renseignements. Mais aussi l'importance d'un véritable travail de fond face à la radicalisation des esprits. Philippe Walkowiak",
                "Depuis la création de la Région bruxelloise, le problème ne cesse d'empirer. Accroissement du nombre de navetteurs, augmentation du parc automobile des Bruxellois rendent la circulation de plus en plus difficile à Bruxelles.Pour améliorer la situation, outre de grands travaux d'aménagements, le gouvernement régional en 1999 a déjà conçu un premier plan régional des déplacements, le plan Iris 1. Aujourd'hui, devant l'évolution de la situation il est urgent de l'adapter. En ce moment, le plan Iris 2 est à l'étude. A terme, il est vraisemblable qu'il faudra trouver des solutions de plus en plus drastiques pour limiter la circulation dans la ville. Le débat est déjà ouvert.",
                "Cette hausse ressort également dans la dernière mise à jour du service public fédéral des Finances, qui indiquait que 2 572 038 déclarations avaient été introduites mercredi à 14h00. À la même période en 2016, 2 484 160 Belges avaient déclaré leurs revenus électroniquement, soit une augmentation de 3,5%.2 484 160 citoyens font leur déclaration eux-mêmes (+8,8%) tandis que 845 518 font appel à un fonctionnaire (-4,1%) pour les aider. Alors que leur délai est fixé au 26 octobre, 182 771 Belges (-0,2%) ont déjà introduit leur déclaration via un comptable. Le porte-parole du SPF Finances, Francis Adyns, est 'très satisfait des chiffres'. Au total, Tax-on-web avait comptabilisé, en 2016, 3,68 millions de déclarations électroniques, un record.",
                "De Tokyo à Washington, de Moscou au Qatar, les responsables et chefs d'Etat ont multiplié les messages de soutien à la France et exprimé leur répulsion. Politiques, têtes couronnées telles la reine Elizabeth II et le roi des Belges, organisations internationales et médias ont rendu hommage aux victimes (12 morts, 11 blessés) de cette 'journée noire pour la liberté d'expression'. 'La France, et la merveilleuse ville de Paris où cette attaque scandaleuse a eu lieu, sont pour le monde une référence intemporelle qui demeurera bien au-delà de la vision haineuse de ces tueurs', a écrit le président américain Barack Obama.",
                "La pandémie du nouveau coronavirus a fait au moins 4.771.320 morts dans le monde depuis que le bureau de l'OMS en Chine a fait état de l'apparition de la maladie fin décembre 2019, selon un bilan établi par l'AFP à partir de sources officielles jeudi à midi. Plus de 233.239.040 cas d'infection ont été officiellement diagnostiqués depuis le début de l'épidémie. Les chiffres se fondent sur les bilans communiqués quotidiennement par les autorités sanitaires de chaque pays. L'OMS estime, en prenant en compte la surmortalité directement et indirectement liée au Covid-19, que le bilan de la pandémie pourrait être deux à trois fois plus élevé que celui officiellement recensé. ",
                "Hier soir, les pays membres de l’EU sont arrivés à la conclusion qu’il ne faut plus décourager nos ressortissants à quitter le pays. Ce matin, un incendie s’est déclaré dans la picine de refroidissement des combustibles usés. Ce soir, le taux de contamination de l’air augmente à Tokyo. Les hélicos qui déversent de l’eau sur le site ont du s’arrêter. Les techniciens aussi. Evidemment, le tremblement de terre de hier à Shizuoka (6 sur Richter mais 4 seulement à Tokyo) et les séismes en chaîne qui ont lieu dans la région de Fukushima ne contribuent en rien à l’amélioration de la situation. La population commence à évacuer Tokyo. Mais sereinement pour l’instant. Dans l’ordre et la discipline. ",
                "Depuis le premier janvier dernier, le cours de l'or a bondi de 18 %. Le lingot d'un kilo vaut actuellement plus de 36 000 euros. De quoi donner des idées à tous ceux qui constatent que leur épargne ne rapporte plus rien. Mais attention, l’or est un placement assez particulier à ne pas mettre entre toutes les mains ! Une première question: pourquoi le prix de l'or est-il à la hausse? L'explication est très simple et très classique: l'or grimpe en même temps que les incertitudes. Elles sont nombreuses aujourd'hui: l'économie patine, les taux d'intérêt sont à zéro, voire négatifs, et les tensions internationales sont grandes. Donc la hausse ne surprend pas les professionnels.",
                "L'ancien président de l'Association Syndicale des Magistrats, Thierry Marchandise, vient d'envoyer une lettre au Ministre de la justice, Koen Geens. Celui qui fut aussi procureur du roi à Charleroi y explique qu'en 45 ans de carrière, il n'a jamais vu une telle déglingue à cause des mesures d'économie imposées par le gouvernement fédéral. Et de donner plusieurs exemples, en vrac : une livraison de 800 rouleaux de ... papier toilette à la Justice de Paix de Gosselies, alors qu'à la Justice de Paix de Charleroi, le personnel est invité à se munir de son propre rouleau. Ou encore, ce magnifique portique détecteur de métal à l'entrée principale du palais de justice carolo."
                ]
text_samples_small = ["Depuis le premier janvier dernier, le cours de l'or a bondi de 18 %. Le lingot d'un kilo vaut actuellement plus de 36 000 euros. De quoi donner des idées à tous ceux qui constatent que leur épargne ne rapporte plus rien. Mais attention, l’or est un placement assez particulier à ne pas mettre entre toutes les mains ! Une première question: pourquoi le prix de l'or est-il à la hausse? L'explication est très simple et très classique: l'or grimpe en même temps que les incertitudes. Elles sont nombreuses aujourd'hui: l'économie patine, les taux d'intérêt sont à zéro, voire négatifs, et les tensions internationales sont grandes. Donc la hausse ne surprend pas les professionnels."]

In [40]:
def predict_probs(texts):
    text_df = pd.DataFrame(texts, columns=['text'])
    textDataset = datasets.Dataset.from_pandas(text_df)
    tokenized_text = textDataset.map(preprocess_function, batched=True)
    tokenized_text = tokenized_text.remove_columns(['text'])
    text_dl = DataLoader(tokenized_text, 
                   shuffle=False,
                   batch_size=batch_size*16,
                   collate_fn=data_collator)
    predictions = []
    for step, batch in enumerate(tqdm(text_dl, total=len(text_dl))):
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)

        with torch.no_grad():        
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        
        predictions += [logit for logit in logits]
        
        b_input_ids.detach().cpu()
        b_input_mask.detach().cpu()
    
    lastLayer = torch.nn.Softmax(dim=1)
    lastLayer.to(device)
    output = lastLayer(torch.tensor(predictions))
    lastLayer.cpu()
    return output.cpu().numpy()

## 3.1 LIME

In [41]:
def cond(val, probas):
    if probas[0] > probas[1]:
        return val < 0
    else:
        return val > 0

if DO_LIME:
    # explain instance with LIME
    #Todo add a split expression
    explainer = LimeTextExplainer(class_names=["Information", "Opinion"], bow=False, random_state=seed, split_expression='\s')
    
    for it, text in enumerate(text_samples):
        text = preprocess(text)
        title = "example_lime_trained_" + str(it+1)
        exp = explainer.explain_instance(text, predict_probs, num_features=20, num_samples=5000)
        exp.local_exp[1] = [(pos, i) if cond(i, exp.predict_proba) else (pos, 0) for pos, i in exp.local_exp[1]]
        #exp.save_to_file("/content/gdrive/My Drive/{}.html".format(title))
    
        if VISUALIZE:
            exp.show_in_notebook(text=text)
        

## 3.2 SHAP (TODO ?)

In [42]:
doShap = False
if doShap:
    #masker = shap.maskers.Text(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s")
    masker = shap.maskers.Text(r"\W")

    explainer = shap.Explainer(predict_probs, masker, output_names=["information", "opinion"]) 
    shap_values = explainer(text_samples_small)
    # visualize the first prediction's explanation for the POSITIVE output
    for elem in shap_values:
      shap.plots.text(elem)
      print("\n\n")

## 3.3 LRP (Based on Chefer's work)

In [43]:
# Load a trained model and vocabulary that you have fine-tuned

model_expl = CamembertForSequenceClassification.from_pretrained("../NeededFiles/camembert_model_punct")
model_expl.eval()

explaner = Generator(model_expl)
tokenizer_expl = AutoTokenizer.from_pretrained("../NeededFiles/camembert_model_punct")

# Copy the model to the GPU.
if torch.cuda.is_available():       
    device = torch.device("cuda")
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
model_expl.to(device)

You are using a model of type camembert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


CamembertForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (add1): Add()
      (add2): Add()
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (matmul1): MatMul()
              (matmul2): MatMul()
              (softmax): Softmax(dim=-1)
 

In [44]:
if EVALUATE:
    train_acc_expl = accuracy_pt(model_expl, train, withGrad=True)
    val_acc_expl = accuracy_pt(model_expl, val, withGrad=True)
    test_acc_expl = accuracy_pt(model_expl, test, withGrad=True)
    
    print(train_acc_expl, train_acc)
    print(val_acc_expl, validation_acc)
    print(test_acc_expl, test_acc)

In [45]:
def retokenize(tokens, expl):#TODO debug this shit 

    last = tokens[0]
    sumExpl = expl[0]
    nbrPart = 1
    flagNext = False
    tokenExplList = []

    for token, expli in [(tokens[i], expl[i]) for i in range(len(tokens))]:
        if token[0] != "<" and token[0] != "▁":
            last += token
            sumExpl += expli
            nbrPart += 1
        else:
            tokenExplList += [(last, sumExpl)]
            last = token
            sumExpl = expli
            nbrPart = 1
            if "</s>" in token:
                break

    tokenExplList += [(token, expli)]
    tokenExplList = tokenExplList[2:]
    

    correctTokList = "".join([token for token, _ in tokenExplList]).split("▁")
    correctTokList = [token for token in correctTokList if token != '']
    expli = [ex for _, ex in tokenExplList]
    tokensFinal = correctTokList
    #print([(tok, exp) for tok,exp in zip(tokensFinal, expli)])
    return tokensFinal, expli



In [52]:
# encode a sentence
#text_batch = ["Premier chiffre : 242 milliards d’euros. Telle est la somme invraisemblable qui repose sur les comptes d’épargne en Belgique. Si on a la curiosité de diviser ce chiffre par dix millions de Belges, ça nous fait tout de même une moyenne de 24.000 € par personne. Si, comme moi, vous avez l’impression de connaître davantage d’individus en-dessous des 24.000 qu’au-dessus, sachez que vous faites partie de la mauvaise moyenne. De toute façon, bonne chance pour aborder le sujet : l’argent qu’on gagne, l’argent qu’on thésaurise est et restera le dernier tabou des Belges. Mais lorsque sortent ces chiffres, comme la semaine dernière, se pose logiquement la question de la manière dont cet argent pourrait être mobilisé et utilisé. C’est vrai que de l’argent fixe, ça n’arrange personne : ni l’Etat, qui se ponctionne lorsque l’argent circule par la TVA, l’IPP, l’ISOC, etc., ni les entreprises, ni même les particuliers, à qui il ne rapporte rien."]
#text_batch = ["De Tokyo à Washington, de Moscou au Qatar, les responsables et chefs d'Etat ont multiplié les messages de soutien à la France et exprimé leur répulsion envers les étrangers. Politiques, têtes couronnées telles la reine Elizabeth II et le roi des Belges, organisations internationales et médias ont rendu hommage aux victimes (12 morts, 11 blessés) de cette 'journée noire pour la liberté d'expression'. 'La France, et la merveilleuse ville de Paris où cette attaque scandaleuse a eu lieu, sont pour le monde une référence intemporelle qui demeurera bien au-delà de la vision haineuse de ces tueurs', a écrit le président américain Barack Obama."]
#text_batch = ["Ah… la Saint-Valentin. Journée annuelle appartenant aux amoureux. Temps merveilleux de ceux et celles qui ne conjuguent plus que par nous, qui s’aiment un peu, beaucoup, passionnément, à la folie. Douces heures que celles du 14 février, moment tendresse où l’on célèbre l’autre, son autre, sa moitié, son presque tout. Si les enthousiastes se presseront au-devant d’étalages tout de roses garnis, à l’affut d’une attention délicate ou d’un présent extravagant chargé d’en mettre plein la vue de l’être aimé, d’autres à l’inverse verront leurs poils se hérisser et s’offusqueront contre ce qu’ils considèreront être une manœuvre capitaliste à peine dissimulée."]

DO_LRP = True
VISUALIZE = True
if DO_LRP:

    text_batch = text_samples

    dict_explanations = {}

    classifications = ["information", "opinion"]
    VISUALIZE = True
    for nbr, itm in enumerate(tqdm(text_batch, total = len(text_batch))):
        model_expl.to("cuda")
        itm = preprocess(itm)  
        encoding = tokenizer_expl(itm, max_length = max_length, padding = 'max_length', truncation=True, return_tensors='pt')
        input_ids = encoding['input_ids'].to("cuda")
        attention_mask = encoding['attention_mask'].to("cuda")

        expl = explaner.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=1)[0]

        # normalize scores
        expl = (expl - expl.min()) / (expl.max() - expl.min())

        # get the model classification
        output = torch.nn.functional.softmax(model_expl(input_ids=input_ids, attention_mask=attention_mask)[1], dim=-1).to("cpu")
        
        classification = output.argmax(dim=-1).item()
        # get class name
        class_name = classifications[classification]
        # if the classification is negative, higher explanation scores are more negative
        # flip for visualization
        if class_name == "opinion":
            expl *= (-1)

        tokens = tokenizer_expl.convert_ids_to_tokens(input_ids.flatten())
        tokensFinal, expli = retokenize(tokens, expl)
        #print(len(tokensFinal), len(expli))

        vis_data_records = [visualization.VisualizationDataRecord(
                                expli,
                                output[0][classification],
                                classification,
                                classification,
                                classification,
                                1,
                                tokensFinal,
                                1)]

        visualization.visualize_text(vis_data_records)

        if VISUALIZE:
            pass
            #visualization.visualize_text(vis_data_records)

        for tok, expl_val in [(tokensFinal[i], expli[i]) for i in range(len(tokensFinal))]:
            word_explanations = dict_explanations.get(tok, (0,0)) 
            word_explanations = word_explanations[0] + 1, word_explanations[1] + expl_val
            dict_explanations[tok] = word_explanations

        print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
    avg_list = [(word, tup[1]/tup[0]) for word, tup in dict_explanations.items()]
    print(sorted(avg_list, key = lambda a: a[1], reverse=True)[1:100])
    print(sorted(avg_list, key = lambda a: a[1])[1:100])

  0%|                                                    | 0/12 [00:00<?, ?it/s]

  8%|███▋                                        | 1/12 [00:00<00:02,  4.29it/s]

Allocated: 39.4 GB
Cached:    40.4 GB


 17%|███████▎                                    | 2/12 [00:00<00:02,  4.66it/s]

Allocated: 39.9 GB
Cached:    40.9 GB


...

Allocated: 44.3 GB
Cached:    45.4 GB


 92%|███████████████████████████████████████▍   | 11/12 [00:02<00:00,  4.79it/s]

Allocated: 44.8 GB
Cached:    45.9 GB


 92%|███████████████████████████████████████▍   | 11/12 [00:02<00:00,  4.25it/s]


RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 47.54 GiB total capacity; 45.66 GiB already allocated; 15.62 MiB free; 46.27 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [38]:
print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Allocated: 17.2 GB
Cached:    18.0 GB


In [33]:
!python -m spacy download fr_core_news_sm
!pip3 install fr_core_news_sm
!pip3 install spacy
!pip3 install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 26.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
import nltk
import math
import regex as re
import pandas as pd
import spacy
import numpy as np
from timeit import default_timer as timer

from matplotlib import pyplot as plt

from spacy.language import Language
#from spacy_lefff import LefffLemmatizer, POSTagger

import json

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [35]:
model_coefs = {
    "pron_1" : 44.0460,
    "pron_2" : 14.6279,
    "nb_on" : 117.0203,
    "pron_rel" : 53.2536,
    "adv_mod" : 96.5657,
    "conj_comp" : 57.9784,
    "nb_vb" : 16.4442,
    "nb_adj" : 21.0758,
    "nb_neg" : 55.1994,
    "nb_interrog" : 10.8113,
    "nb_exclam" : 10.1036,
    "nb_pointvirg" : 8.9525,
    "nb_deuxpoints" : 3.7551,
    "nb_susp" : 0.7113,
    "nb_tirets" : 3.0284,
    "nb_parenth" : 0.1402,
    "nb_long_words" : 21.4322,
    "nb_digits" : 67.9309,
    "lexique3_sentiment" : 0.2590,
    "nrc_sentiment" : 7.0299    
}

# insert filepaths to each sentiment lexicon
#TODO add lexicons in NeededFiles
lexique3_path = f"NeededFiles/lexicons/open_lexicon.csv"
nrc_path = f"NeededFiles/lexicons/nrc_lexicon.json"

nlp = spacy.load("fr_core_news_sm")

columns = ["Word", "Pol.Val.", "Catégories"]
lexique3_df = pd.read_csv(lexique3_path, sep=';', error_bad_lines=False, usecols = columns)
lexique3_df.set_index("Word", inplace=True)
lexique3_dict = lexique3_df.to_dict()
lexique3_dict = lexique3_dict["Pol.Val."]

# load NRC lexicon

with open(nrc_path,'r') as f:
    data = json.loads(f.read())

nrc_df = pd.json_normalize(data, record_path =['LexicalEntry'])
nrc_df = nrc_df[['Lemma.-writtenForm', 'Sense.Sentiment.-polarity']].copy()
nrc_df.set_index("Lemma.-writtenForm", inplace=True)
nrc_df_pos = nrc_df[nrc_df['Sense.Sentiment.-polarity']=="positive"]
nrc_df_neg = nrc_df[nrc_df['Sense.Sentiment.-polarity']=="negative"]
nrc_lex_df = pd.concat([nrc_df_pos, nrc_df_neg])
nrc_dict = nrc_lex_df.to_dict()
nrc_dict = nrc_dict["Sense.Sentiment.-polarity"]

lexique3_lex = list(lexique3_dict.keys())
nrc_lex = list(nrc_dict.keys())

def clean_html(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

# takes str in argument and returns list of tokens (using nltk tokenizer)
nltk.download('punkt')
def tokenize(art):

    art_clean = re.sub(r'(?<=\p{L})(\.)(?=\p{L})', ". ", art)
    art_clean = re.sub(r"(?<=\p{L})(')(?=\p{L})", "' ", art_clean)

    words = nltk.word_tokenize(art_clean, "french")
    tokens = []
    for word in words:
        lword = word.lower()
        tokens.append(lword)
    return tokens

# takes list of tokens in argument and returns list of tokens without punctuation marks
def remove_punctuation(tokens):
    puncts = [".", ",", "!", "?", "’", "(", ")", "''", "...", "``", "'", "%", "“", ":", "-", ";"]
    lex_tokens = []
    for token in tokens:
        if token not in puncts:
            lex_tokens.append(token)

    return lex_tokens

# turns article (str) into object that can be lemmatized/pos-tagged with spacy
def spacy_art(art):
    return nlp(art)

# takes article (turned into spacy object) and returns list of pos-tags (not tokens)
def pos_tags(spacyd_art):

    tags = []
    for a in spacyd_art:
        tags.append(a.tag_)

    return tags

# takes article (turned into spacy object) and returns list of lemmatized tokens
def lemmatize(spacyd_art):

    lemmas = []
    for a in spacyd_art:
        lemmas.append(a.lemma_)

    lemmas = remove_punctuation(lemmas)
    return lemmas

prons_1 = ["moi", "je", "me", "nous", "mien", "miens", "nos", "notre", "nôtres", "mon", "ma", "mes", "-moi", "-nous"]
prons_2 = ["toi", "tu", "te", "vous", "tien", "tiens", "vos", "votre", "vôtres", "ton", "ta", "tes", "-toi", "-vous"]
pr_rel = ["qui", "que", "dont", "où", "lequel", "quoi", "qu'"]
v_list = ["V", "VINF", "VIMP", "VS", "VERB"]
conj_comp_l = ["or", "donc", "car", "pourtant", "cependant", "néanmoins", "toutefois", "malgré", "outre", "quoique", "tandis"]
adv_mod_l = ["vraiment", "réellement", "sûrement", "certainement", "assurément", "incontestablement", "sans doute", "sans aucun doute", "heureusement", "malheureusement", "peut-être"]
negs = ["ne", "ni", "n", "non", "aucun", "sans", "nul", "nulle", "n'", "n’"]

def count_pron_1(tokens):
    prons_1 = ["moi", "je", "me", "nous", "mien", "miens", "nos", "notre", "nôtres", "mon", "ma", "mes"]

    cnt = 0
    for token in tokens:
        if token in prons_1:
            cnt += 1
            
    if len(tokens) > 0:
        return cnt/len(tokens)
    else:
        return 0
    
def count_pron_2(tokens):
    prons_2 = ["toi", "tu", "te", "vous", "tien", "tiens", "vos", "votre", "vôtres", "ton", "ta", "tes"]

    cnt = 0
    for token in tokens:
        if token in prons_2:
            cnt += 1

    if len(tokens) > 0:
        return cnt/len(tokens)
    else:
        return 0
    
def count_on(tokens):
    
    cnt = tokens.count("on")
    
    if len(tokens) > 0:
        return cnt/len(tokens)
    else:
        return 0

# takes list of tokens and returns count of relative pronouns (float)
def count_pron_rel(tokens):
    pr_rel = ["qui", "que", "dont", "où", "lequel", "quoi"]

    cnt = 0
    for token in tokens:
        if token in pr_rel:
            cnt += 1

    if len(tokens) > 0:
        return cnt/len(tokens)
    else:
        return 0

# takes list of pos-tags and returns count of adjectives (float)
def count_adj(tags):

    cnt = tags.count("ADJ")

    nb_punct = tags.count("PONCT")
    nb_tags = len(tags) - nb_punct

    if nb_tags == 0:
        return 0
    else:
        return cnt/nb_tags

# takes list of pos-tags and returns count of verbs (float)
def count_vb(tags):
    v_list = ["V", "VINF", "VIMP", "VS", "VERB"]

    cnt = 0
    for tag in tags:
        if tag in v_list:
            cnt += 1

    nb_punct = tags.count("PONCT")
    nb_tags = len(tags) - nb_punct

    if nb_tags == 0:
        return 0
    else:
        return cnt/nb_tags
    
def count_adv_mod(tokens):
    adv_mod = ["vraiment", "réellement", "sûrement", "certainement", "assurément", "incontestablement", "sans doute", "sans aucun doute", "heureusement", "malheureusement", "peut-être"]
    
    cnt = 0
    for token in tokens:
        if token in adv_mod:
            cnt += 1

    if len(tokens) > 0:
        return cnt/len(tokens)
    else:
        return 0

def count_conj_comp(tokens):
    conj_comp = ["or", "donc", "car", "pourtant", "cependant", "néanmoins", "toutefois", "malgré", "outre", "quoique", "tandis"]
    
    cnt = 0
    for token in tokens:
        if token in conj_comp:
            cnt += 1

    if len(tokens) > 0:
        return cnt/len(tokens)
    else:
        return 0   

# takes list of tokens and returns count of negation words (float)
def count_neg(tokens):
    negs = ["ne", "ni", "n", "non", "aucun", "sans", "nul", "nulle"]

    cnt = 0
    for token in tokens:
        if token in negs:
            cnt += 1

    if len(tokens) > 0:
        return cnt/len(tokens)
    else:
        return 0
    
def length_words(tokens):

    tot_length = 0
    for token in tokens:
        tot_length += len(token)

    if len(tokens) > 0:
        return tot_length / len(tokens)
    else:
        return 0
    
# takes list of tokens and returns count of interrogation marks (float)
def count_interrog(tokens):
    final_punct = [".", "?", "!"]

    cnt_interrog = 0
    cnt_finalp = 0
    for token in tokens:
        if token == "?":
            cnt_interrog += 1
        if token in final_punct:
            cnt_finalp += 1

    if cnt_finalp <= 0:
        return 0
    else:
        return cnt_interrog / cnt_finalp

# takes list of tokens and returns count of exclamation marks (float)
def count_exclam(tokens):
    final_punct = [".", "?", "!"]

    cnt_exclam = 0
    cnt_finalp = 0
    for token in tokens:
        if token == "!":
            cnt_exclam += 1
        if token in final_punct:
            cnt_finalp += 1

    if cnt_finalp <= 0:
        return 0
    else:
        return cnt_exclam / cnt_finalp

# takes list of tokens and returns count of semicolons (float)
def count_pointvirg(tokens):
    sep_punct = [".", "?", "!", ";", ":", "—"]

    cnt_pointvirg = 0
    cnt_sepp = 0
    for token in tokens:
        if token == ";":
            cnt_pointvirg += 1
        if token in sep_punct:
            cnt_sepp += 1

    if cnt_sepp <= 0:
        return 0
    else:
        return cnt_pointvirg / cnt_sepp

# takes list of tokens and returns count of colons (float)
def count_deuxpoints(tokens):
    sep_punct = [".", "?", "!", ";", ":", "..."]

    cnt_deuxpoints = 0
    cnt_sepp = 0
    for token in tokens:
        if token == ":":
            cnt_deuxpoints += 1
        if token in sep_punct:
            cnt_sepp += 1

    if cnt_sepp <= 0:
        return 0
    else:
        return cnt_deuxpoints / cnt_sepp
    
def count_tirets(tokens):  
    sep_punct = [".", "?", "!", ";", ":", "...", "("]
    
    cnt_tirets = 0
    cnt_sepp = 0
    for token in tokens:
        if token == "—":
            cnt_tirets += 1
        if token in sep_punct:
            cnt_sepp += 1
    cnt_tirets = cnt_tirets/2    # diviser par 2 car les tirets vont souvent par paire
    
    if cnt_sepp <= 0:
        return 0
    else:
        return cnt_tirets / cnt_sepp

# takes list of tokens and returns count of ellipsis points (float)
def count_susp(tokens):
    sep_punct = [".", "?", "!", ";", ":", "..."]

    cnt_susp = 0
    cnt_sepp = 0
    for token in tokens:
        if token == "...":
            cnt_susp += 1
        if token in sep_punct:
            cnt_sepp += 1

    if cnt_sepp <= 0:
        return 0
    else:
        return cnt_susp / cnt_sepp
    
def count_parenth(clean_article):    # renvoie le nombre de paires de parenthèses
    cnt = len(re.findall(r'\(([^)]+)\)', clean_article))
    return cnt

def count_citations(clean_article):
    cnt1 = len(re.findall(r'"(.*?)"', clean_article))
    cnt2 = len(re.findall(r'«(.*?)»', clean_article))
    cnt3 = len(re.findall(r'“(.*?)”', clean_article))
    cnt = cnt1 + cnt2 + cnt3

    return cnt  

def count_long_words(tokens):
    
    cnt_long_words = 0
    
    for i in tokens:
        if len(i) > 8:
            cnt_long_words += 1
    
    if len(tokens) <= 0:
        return 0
    else:
        return cnt_long_words / len(tokens)

def count_digits(tokens):
    cnt = 0
    for token in tokens:
        has_digit = False
        for char in token:
            if char.isdigit():
                has_digit = True
        if has_digit:
            if not re.match("ovid", token):
                cnt += 1

    if len(tokens) <= 0:
        return 0
    else:
        return cnt / len(tokens)

def lexique3_score(lemmas):
    negative_c = 0
    positive_c = 0

    for lemma in lemmas:
        if lemma in lexique3_dict:
            if lexique3_dict[lemma] == 'neg':
                negative_c += 1
                #print(lemma, lexique3_dict[lemma])
                
            if lexique3_dict[lemma] == 'positif':
                positive_c += 1
                #print(lemma, lexique3_dict[lemma])
    
    if len(lemmas) == 0:
        positive_cnt = 0
        negative_cnt = 0
        sentiment_cnt = 0
    else:
        positive_cnt = positive_c/len(lemmas)
        negative_cnt = negative_c/len(lemmas)
        sentiment_cnt = (positive_c + negative_c)/len(lemmas)
        
    return (positive_cnt, negative_cnt, sentiment_cnt)

def nrc_score(lemmas):
    negative_c = 0
    positive_c = 0

    for lemma in lemmas:
        if lemma in nrc_dict:
            if nrc_dict[lemma] == 'negative':
                negative_c += 1
                #print(lemma, nrc_dict[lemma])
                
            if nrc_dict[lemma] == 'positive':
                positive_c += 1
                #print(lemma, nrc_dict[lemma])

    if len(lemmas) == 0:
        positive_cnt = 0
        negative_cnt = 0
        sentiment_cnt = 0
    else:
        positive_cnt = positive_c/len(lemmas)
        negative_cnt = negative_c/len(lemmas)
        sentiment_cnt = (positive_c + negative_c)/len(lemmas)

    return (positive_cnt, negative_cnt, sentiment_cnt)

def textblob_sent_subj(cleaned_art):
    
    blob = tb(cleaned_art)
    return blob.sentiment

def cttr(tokens):

    types = []
    for token in tokens:
        if token not in types:
            types.append(token)
    if len(tokens) == 0:
        return 0
    else:
        cttr = len(types) / math.sqrt(2 * len(tokens))
        return cttr

def score_subj(article):
    
    article = clean_html(article)
    tokens = tokenize(article)
    unpunct_tokens = remove_punctuation(tokens)
    spacyd_article = spacy_art(article)
    tags = pos_tags(spacyd_article)
    lemmas = lemmatize(spacyd_article)
    
    score = 0
    
    pron_2 = count_pron_2(unpunct_tokens) * model_coefs["pron_2"]
    nb_on = count_on(unpunct_tokens) * model_coefs["nb_on"]
    pron_rel = count_pron_rel(unpunct_tokens) * model_coefs["pron_rel"]
    adv_mod = count_adv_mod(unpunct_tokens) * model_coefs["adv_mod"]
    conj_comp = count_conj_comp(unpunct_tokens) * model_coefs["conj_comp"]
    nb_adj = count_adj(tags) * model_coefs["nb_adj"]
    nb_neg = count_neg(unpunct_tokens) * model_coefs["nb_neg"]
    nb_exclam = count_exclam(tokens) * model_coefs["nb_exclam"]
    nb_interrog = count_interrog(tokens) * model_coefs["nb_interrog"]
    nb_pointvirg = count_pointvirg(tokens) * model_coefs["nb_pointvirg"]
    nb_deuxpoints = count_deuxpoints(tokens) * model_coefs["nb_deuxpoints"]
    nb_tirets = count_tirets(tokens) * model_coefs["nb_tirets"]
    nb_parenth = count_parenth(article) * model_coefs["nb_parenth"]
    lexique3_sentiment = lexique3_score(lemmas)[2] * model_coefs["lexique3_sentiment"]
    nrc_sentiment = nrc_score(lemmas)[2] * model_coefs["nrc_sentiment"]
    
    pron_1 = count_pron_1(unpunct_tokens) * model_coefs["pron_1"]
    nb_vb = count_vb(tags) * model_coefs["nb_vb"]
    nb_susp = count_susp(tokens) * model_coefs["nb_susp"]
    nb_long_words = count_long_words(tokens) * model_coefs["nb_long_words"]
    nb_digits = count_digits(unpunct_tokens) * model_coefs["nb_digits"]

    
    for_opi = pron_2 + nb_on + pron_rel + adv_mod + conj_comp + nb_neg + nb_adj + lexique3_sentiment + nrc_sentiment + nb_exclam + nb_interrog + nb_pointvirg + nb_deuxpoints + nb_tirets + nb_parenth
    score += for_opi
    for_info = nb_digits + nb_vb + nb_long_words + pron_1 + nb_susp
    score -= for_info
    
    return score

# define limit score_subj between opinion and information

threshold = 0.39

def create_map(article):
    
    vmap = []
    
    article = clean_html(article)
    article_space = space_punct(article)
    tokens = re.split("\s+", article_space)
    tokens = list(filter(None, tokens))
    unpunct_tokens = remove_punctuation(tokens)
    spacyd_article = spacy_art(article_space)
    tags = pos_tags(spacyd_article)
    lemmas = lemmatize(spacyd_article)
    
    score = score_subj(article_space)

    tag_lemma_dict = {}

    for sp in spacyd_article:
      tag_lemma_dict[sp.text] = (sp.tag_, sp.lemma_)
    
    if score > threshold:
        
        classif = "OPINION"

        pron_2 = count_pron_2(unpunct_tokens) * model_coefs["pron_2"]
        nb_on = count_on(unpunct_tokens) * model_coefs["nb_on"]
        pron_rel = count_pron_rel(unpunct_tokens) * model_coefs["pron_rel"]
        adv_mod = count_adv_mod(unpunct_tokens) * model_coefs["adv_mod"]
        conj_comp = count_conj_comp(unpunct_tokens) * model_coefs["conj_comp"]
        nb_adj = count_adj(tags) * model_coefs["nb_adj"]
        nb_neg = count_neg(unpunct_tokens) * model_coefs["nb_neg"]
        nb_exclam = count_exclam(tokens) * model_coefs["nb_exclam"]
        nb_interrog = count_interrog(tokens) * model_coefs["nb_interrog"]
        nb_pointvirg = count_pointvirg(tokens) * model_coefs["nb_pointvirg"]
        nb_deuxpoints = count_deuxpoints(tokens) * model_coefs["nb_deuxpoints"]
        nb_tirets = count_tirets(tokens) * model_coefs["nb_tirets"]
        nb_parenth = count_parenth(article) * model_coefs["nb_parenth"]
        lexique3_sentiment = lexique3_score(lemmas)[2] * model_coefs["lexique3_sentiment"]
        nrc_sentiment = nrc_score(lemmas)[2] * model_coefs["nrc_sentiment"]
        
        for word in tokens:
            
            attention = 0

            if word in tag_lemma_dict:

                tag = tag_lemma_dict[word][0]
                lemma = tag_lemma_dict[word][1]

                if word in prons_2:
                    attention += pron_2
                if word == "on" or word == "-on":
                    attention += nb_on
                if word in pr_rel:
                    attention += pron_rel
                if word in adv_mod_l:
                    attention += adv_mod
                if word in conj_comp_l:
                    attention += conj_comp
                if tag == "ADJ":
                    attention += nb_adj
                if word in negs:
                    attention += nb_neg
                if "!" in word:
                    attention += nb_exclam
                if "?" in word:
                    attention += nb_interrog
                if ";" in word:
                    attention += nb_pointvirg
                if ":" in word:
                    attention += nb_deuxpoints
                if "—" in word:
                    attention += nb_tirets
                if "(" in word or ")" in word:
                    attention += nb_parenth
                if lemma in lexique3_lex:
                    attention += lexique3_sentiment
                if lemma in nrc_lex:
                    attention += nrc_sentiment
                            
            vmap.append(attention)
        
    else:
        classif = "INFORMATION"
        
        pron_1 = count_pron_1(unpunct_tokens) * model_coefs["pron_1"]
        nb_vb = count_vb(tags) * model_coefs["nb_vb"]
        nb_susp = count_susp(tokens) * model_coefs["nb_susp"]
        nb_long_words = count_long_words(tokens) * model_coefs["nb_long_words"]
        nb_digits = count_digits(unpunct_tokens) * model_coefs["nb_digits"]

        for word in tokens:
            
            attention = 0

            if word in tag_lemma_dict:

                tag = tag_lemma_dict[word][0]
                lemma = tag_lemma_dict[word][1]
                            
                if word in prons_1:
                    attention += pron_1
                if tag in v_list:
                    attention += nb_vb
                if "..." in word:
                    attention += nb_susp
                if len(word) > 8:
                    attention += nb_long_words
                if tag == "NUM":
                    attention += nb_digits
            
            vmap.append(attention)
            
    print(classif)
    
    xmin = min(vmap) 
    xmax = max(vmap)
    for i, x in enumerate(vmap):
        vmap[i] = (x-xmin) / (xmax-xmin)
    
    return vmap

def colorize(words, color_array):
    cmap=matplotlib.cm.Reds
    template = '<span class="barcode"; style="color: black; background-color: {}">{}</span>'
    colored_string = ''
    for word, color in zip(words, color_array):
        color = matplotlib.colors.rgb2hex(cmap(color)[:3])
        #print(color)
        colored_string += template.format(color, '&nbsp' + word + '&nbsp')
    return colored_string

The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.




FileNotFoundError: [Errno 2] No such file or directory: 'NeededFiles/lexicons/open_lexicon.csv'

In [ ]:
def space_punct(article):

  regexp1 = re.compile('\?')
  regexp2 = re.compile('\!')
  regexp3 = re.compile('\.')
  regexp4 = re.compile('\,')
  regexp5 = re.compile('\;')
  regexp6 = re.compile('\"')
  regexp7 = re.compile('\(')
  regexp8 = re.compile('\)')
  regexp9 = re.compile("\'")
  regexp10 = re.compile("\’")

  processed_article = re.sub(regexp1, ' ? ', article)
  processed_article = re.sub(regexp2, ' ! ', processed_article)
  processed_article = re.sub(regexp3, ' . ', processed_article)
  processed_article = re.sub(regexp4, ' , ', processed_article)
  processed_article = re.sub(regexp5, ' ; ', processed_article)
  processed_article = re.sub(regexp6, ' " ', processed_article)
  processed_article = re.sub(regexp7, ' ( ', processed_article)
  processed_article = re.sub(regexp8, ' ) ', processed_article)
  processed_article = re.sub(regexp9, " ' ", processed_article)
  processed_article = re.sub(regexp10, " ’ ", processed_article)


  return processed_article

In [ ]:
# testing linguistic attention map

art = "Moad est ce jeune molenbeekois de 14 ans, tabassé par des policiers qui l'auraient confondu avec des délinquants auteurs de vols. Nombreuses plaies au visage, choc psychologique, famille scandalisée par cette agression sans doute à caractère raciste et anti-jeunes sur un adolescent qui se serait trouvé au mauvais endroit au mauvais moment."

text = space_punct(art)
print(text)

tkns = re.split("\s+", text)
tkns = list(filter(None, tkns))

test_map = create_map(art)

color_test = colorize(tkns, test_map)
display(HTML(color_test))

print(len(tkns), len(test_map))

In [ ]:
# create attention map vector from lime explanation ; exp = LIME-explained instance
def lime_to_map(exp):

  lime_tuplist = list(exp.as_map().values())[0]
  lime_tuplist.sort()

  lime_map = []
  for t in lime_tuplist:
      lime_map.append(t[1])

  return lime_map

In [ ]:
# compute simple euclidian distance between two attention maps (vectors)
def euclidian_dist(map1, map2):

  if len(map1) != len(map2):
    return 'ERROR : the vector maps must be of same size (based on same text).'

  array1 = np.array(map1)
  array2 = np.array(map2)

  dist = np.linalg.norm(array1 - array2)

  return dist

# compute correlation between two attention maps 
from scipy.stats.stats import pearsonr 

def correlation_maps(map1, map2):

    return pearsonr(map1, map2)

# compute overlap in word selections ("behavioral similarity"), counting ALSO un-attentioned words (where attention = 0)
# requires list of tokens of base text + 2 attention map vectors
# returns tuple : (overlap_rate, overlapping_words_list)

def word_overlap_broad(token_list, map1, map2):

  cnt_overlap = 0
  overlap_list = []

  if len(map1) != len(map2) or len(map1) != len(token_list) or len(map2) != len(token_list):
    return 'ERROR : the vector maps the and token list must be of same size (based on same text).'
  else:
    for i in range(len(map1)-1):
            
        if map1[i] == 0.0 and map2[i] == 0.0:
            cnt_overlap += 1
        if map1[i] > 0.0 and map2[i] > 0.0:
            cnt_overlap += 1
            overlap_list.append(token_list[i])

    total_overlap = cnt_overlap/len(map1)

    return (total_overlap, overlap_list)

# compute overlap in word selections ("behavioral similarity"), counting ONLY attentioned words (where attention > 0)
# requires list of tokens of base text + 2 attention map vectors
# returns tuple : (overlap_rate, overlapping_words_list)

def word_overlap_narrow(token_list, map1, map2):
  
  cnt_overlap = 0
  cnt_attent = 0
  overlap_list = []

  if len(map1) != len(map2) or len(map1) != len(token_list) or len(map2) != len(token_list):
    return 'ERROR : the vector maps the and token list must be of same size (based on same text).'
  else:
    for i in range(len(map1)-1):

      if map1[i] > 0.0 and map2[i] > 0.0:
          cnt_overlap += 1
          cnt_attent += 1
          overlap_list.append(token_list[i])
      if map1[i] > 0.0 and map2[i] == 0.0:
          cnt_attent += 1
      if map1[i] == 0.0 and map2[i] > 0.0:
          cnt_attent += 1

    total_overlap = cnt_overlap/cnt_attent

    return (total_overlap, overlap_list)

# compute lexical similarity (distribution over lexical categories), euclidian distance between attentioned pos-tags
# requires list of tags of base text + 2 attention map vectors

def lexical_similarity(tags_list, map1, map2):

  tags1 = {}
  tags2 = {}

  if len(map1) != len(map2) or len(map1) != len(tags_list) or len(map2) != len(tags_list):
    return 'ERROR : the vector maps the and tags list must be of same size (based on same text).'
  else:
    for i in range(len(map1)-1):

      if map1[i] > 0.0:
          
          tag = tags_list[i]
          if tag in tags1:
              tags1[tag] += 1
          else:
              tags1[tag] = 1
              
      if map2[i] > 0.0:
          
          tag = tags_list[i]
          if tag in tags2:
              tags2[tag] += 1
          else:
              tags2[tag] = 1

    tags1_df = pd.DataFrame(tags1.items(), columns=['tag', 'cnt_info'])
    tags2_df = pd.DataFrame(tags2.items(), columns=['tag', 'cnt_opi'])
    tags_df = pd.merge(tags1_df, tags2_df, on=["tag"], how="outer")
    tags_df = tags_df.fillna(0.0)

    dist = np.linalg.norm(tags_df["cnt_opi"].values - tags_df["cnt_info"].values)
    return dist

In [ ]:
text_samples = ["Moad est ce jeune molenbeekois de 14 ans, tabassé par des policiers qui l'auraient confondu avec des délinquants auteurs de vols. Nombreuses plaies au visage, choc psychologique, famille scandalisée par cette agression sans doute à caractère raciste et anti-jeunes sur un adolescent qui se serait trouvé au mauvais endroit au mauvais moment."]
comp_d = {}

lime_explainer = LimeTextExplainer(class_names=["Information", "Opinion"], bow=False, split_expression="\s+")

for it, text in enumerate(text_samples):

  d = {}
  d["text"] = text

  text = space_punct(text)

  tkns = re.split("\s+", text)
  tkns = list(filter(None, tkns))

  ling_map = create_map(text)
  color_test = colorize(tkns, ling_map)
  display(HTML(color_test))
  d["ling"] = ling_map

  exp = lime_explainer.explain_instance(text, predict_probs, num_features=len(tkns)//2, num_samples=3000)
  exp.show_in_notebook(text=text)
  lime_map = lime_to_map(exp)
  d["lime"] = lime_map

  lrp_map = None
  d["lrp"] = lrp_map

  comp_d[it] = d

In [ ]:
print(len(comp_d[0]['lime']))
print(len(comp_d[0]['ling']))